# Multiple Dispatch 
## Basics

"Any" is at the top of the type hierarchy in Julia. For this example, we'll create a generic function and start specifying types that are more and more specific.

In [ ]:
f(x) = "Any"
f(x::Int) = "Int"

We'll test the function with a few examples.

In [ ]:
f("two")

In [ ]:
f(2.0)

In [ ]:
f(2)

"Multiple dispatch," as opposed to single dispatch, means that all of the parameter's types are taken into account and not just the most specific type.

We'll create two more methods for our function. These methods rely on combinations of increasingly specific types.

In [ ]:
f(x::Int,y) = "Int and Any"
f(x::Int,y::AbstractString) = "Int and String"

We'll test with a few more examples.

In [ ]:
f(2,sum)

In [ ]:
f(2,"1")

And here's the summary of our function with all of its methods.

In [ ]:
methods(f)

## Using multiple dispatch

Let's say we want to write a function that can take as input a matrix that can be either full or sparse. In a language that *doesn't* use multiple dispatch, we could write it this way:

In [ ]:
function foo(A)
    if isparse(A)
        "Sparse"
    else
        "Full"
    end
end

... or write it this way...

In [ ]:
foo(A) = "Full"
foo_sparse(A) = "Sparse"

In a traditional object oriented language we would add methods to the matrix and sparse matrix classes. In Julia we can define them this way. (Note: "CSC" stands for "Compressed Sparse Column.")

In [ ]:
foo(A::Matrix) = "Full"
foo(A::SparseMatrixCSC) = "Sparse"

Testing with examples of full and sparse matrices.

In [ ]:
foo(rand(10,10))

In [ ]:
foo(sparse(rand(10,10)))

## Collecting symbols in a tree

This example that makes heavy use of dispatch combined with recursion. Let's say we want to parse a Julia file and collect all the variable names, which could then be used to provide autocompletion in a text editor.

In Julia an expression is a *tree* – an Abstract Syntax Tree, or AST — that contains a piece of code. Children in the tree can be other expressions, symbols, or meta-data such as line numbers.

For this example, we will first create a tree by defining an expression. Second, we will traverse the tree and collect all the symbols it contains.

### Defining an expression

In [ ]:
ex = :(y = sin(x))

### Creating the tree

We'll use `dump()` to display the tree. In this tree, most of the leaves are symbols.

In [ ]:
dump(ex)

### Tree searching function

Now we can define a function that takes as its argument the expression we created. We then call the function recursively until it finds a symbol, which it will collect in an array.

In [ ]:
function symbols(ex::Expr)
    out = Symbol[]
    for child in ex.args
        s = symbols(child)
        collect_symbols!(out,s)
    end
    out
end

If the function finds a symbol, then return it. If it finds anything else, then return nothing.

In [ ]:
symbols(s::Symbol) = s
symbols(s::Any) = nothing

If we get a symbol we simply add it to the output. If we get an array of symbols, then we splice it to output so we don't get nested arrays. And if we get nothing, we do nothing.

In [ ]:
collect_symbols!(out,s::Symbol) = push!(out,s)
collect_symbols!(out,s::Array{Symbol,1}) = push!(out,s...)
collect_symbols!(out,s::Void) = nothing

### Results of the search

Reminder: The original expressions was y = sin(x).

In [ ]:
symbols(ex)